In [ ]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append("/home/joosep/particleflow/mlpf")
import tfmodel.model
import tfmodel.data
import tfmodel.model_setup

import yaml
import matplotlib.pyplot as plt
import matplotlib 
import os
os.chdir("/home/joosep/particleflow")

import pandas
import networkx
import glob

In [ ]:
with open("/home/joosep/particleflow/parameters/cms-gnn-dense.yaml") as f:
    config = yaml.load(f)
config["setup"]["multi_output"] = True

In [ ]:
model = tfmodel.model_setup.make_gnn_dense(config, tf.float32)

In [ ]:
cds = config["dataset"]

dataset_def = tfmodel.data.Dataset(
    num_input_features=int(cds["num_input_features"]),
    num_output_features=int(cds["num_output_features"]),
    padded_num_elem_size=6400,
    raw_path=cds.get("raw_path", None),
    raw_files=cds.get("raw_files", None),
    processed_path=cds["processed_path"],
    validation_file_path="data/TTbar_14TeV_TuneCUETP8M1_cfi/val/pfntuple_*.pkl.bz2",
    schema=cds["schema"]
)

dataset_transform = tfmodel.model_setup.targets_multi_output(config['dataset']['num_output_classes'])

In [ ]:
Xs = []
ygens = []
ycands = []

for fi in dataset_def.val_filelist[:1]:
    print(fi)
    X, ygen, ycand = dataset_def.prepare_data(fi)

    Xs.append(np.concatenate(X))
    ygens.append(np.concatenate(ygen))
    ycands.append(np.concatenate(ycand))

X_val = np.concatenate(Xs)
ygen_val = np.concatenate(ygens)
ycand_val = np.concatenate(ycands)

X_val, ycand_val, _ = dataset_transform(X_val, ycand_val, None)


In [ ]:
ret = model(X_val[:1])
model.load_weights("/home/joosep/particleflow/experiments/cms-gnn-dense-2cc4e7f9.gpu0.local/weights-500-40.204285.hdf5")
ret = model.predict(X_val, batch_size=10)

In [ ]:
x = X_val[0]
msk = x[:, 0] == 8

In [ ]:
training = False
msk = X_val[:, :, 0] != 0
n_elems = np.sum(msk, axis=1)
enc = model.enc(X_val)
enc_id = model.activation(model.ffn_enc_id(enc))
enc_reg = model.activation(model.ffn_enc_reg(enc))

enc_id1 =  model.cg_id[0](enc_id, msk, training)
enc_id2 =  model.cg_id[1](enc_id1, msk, training)

x_dist = model.cg_id[0].ffn_dist(enc_id)
bins_split1, x_binned1, dm1, msk_binned1 = model.cg_id[0].dist(x_dist, enc_id, msk)

x_dist = model.cg_reg[0].ffn_dist(enc_reg)
bins_split2, x_binned2, dm2, msk_binned2 = model.cg_reg[0].dist(x_dist, enc_reg, msk)

In [ ]:
def get_bin_index(bs):
    bin_index = []

    for ielem in range(6400):
        if X_val[0, ielem, 0] != 0:
            for ibin in range(bins_split.shape[1]):
                if ielem in bs[ibin]:
                    bin_index.append(ibin)
                    break
        else:
            break
    return bin_index

In [ ]:
msk = X_val[0][:, 0] != 0
eta = X_val[0][msk, 2]
phi = X_val[0][msk, 3]
typ = X_val[0][msk, 0]
energy = X_val[0][msk, 4]

evenly_spaced_interval = np.linspac
e(0, 1, 10)
colorlist = [cm.rainbow(x) for x in evenly_spaced_interval]
bin_idx = get_bin_index(bins_split1[0].numpy())

plt.figure(figsize=(4,4))
plt.scatter(eta, phi, c=[colorlist[bi] for bi in bin_idx], marker=".")
plt.xlabel("eta")
plt.ylabel("phi")
plt.title("Binning in classification layer 1")
plt.savefig("bins_cls_layer1.pdf")

In [ ]:
bin_idx = get_bin_index(bins_split2[0].numpy())

plt.figure(figsize=(4,4))
plt.scatter(eta, phi, c=[colorlist[bi] for bi in bin_idx], marker=".")
plt.xlabel("eta")
plt.ylabel("phi")
plt.title("Binning in regression layer 1")
plt.savefig("bins_reg_layer1.pdf")

In [ ]:
dmn = dm[0].numpy()
for i in range(dmn.shape[0]):
    fig = plt.figure(figsize=(5,4))
    plt.imshow(dmn[i], interpolation="none", norm=matplotlib.colors.LogNorm(vmin=1e-4, vmax=1), cmap="Blues")
    plt.colorbar()
    plt.title("Learned graph structure, bin {}".format(i))
    plt.xlabel("elem index $i$")
    plt.ylabel("elem index $j$")
    plt.savefig("dm_{}.pdf".format(i))

In [ ]:
# for iweight in range(1,501):
#     wf = glob.glob("/home/joosep/particleflow/experiments/cms-gnn-skipconn-v2-b7e87c83/weights-{:02d}-*.hdf5".format(iweight))
#     model.load_weights(wf[0])
    
#     iev = 1

#     X = X_val[iev]
#     msk = X[:, 0] != 0
#     X = X[msk]

#     ret = model(X_val[iev:iev+1])
    
#     dm1 = tf.sparse.to_dense(ret["dm"]).numpy()
    
#     df = pandas.DataFrame(X[:, [0,2,3,4,9,10,11,12]], columns=["type", "eta", "phi", "energy", "eta_ecal", "phi_ecal", "eta_hcal", "phi_hcal"])
    
#     g = networkx.DiGraph()
#     for irow in range(len(df)):
#         g.add_node(irow, **{
#             k: df.loc[irow, k] for k in df.columns
#         })

#     rows, cols = dm1[0].nonzero()

#     for irow, icol in zip(rows, cols):
#         if irow < len(df) and icol < len(df):
#             g.add_edge(irow, icol, weight=dm1[0][irow, icol])

#     pos = {
#         i: (g.nodes[i]["eta"], g.nodes[i]["phi"]) for i in g.nodes
#     }
#     colors = [
#         colorlist[int(g.nodes[i]["type"])] for i in g.nodes
#     ]
    
#     draw_node(500)
#     plt.title(iweight)
#     plt.savefig("node_500_iter_{:03d}.png".format(iweight), facecolor="w")
#     plt.clf()
    
#     plt.figure(figsize=(10,10))
#     plt.title(iweight)
#     plt.imshow(dm1[0][:len(X), :len(X)], interpolation="none", cmap="Blues", norm=matplotlib.colors.Normalize(vmin=0, vmax=1))
#     plt.colorbar()
#     plt.savefig("dm_iter_{:03d}.png".format(iweight), facecolor="w")
#     plt.clf()

#     draw_prediction(ret, iev)
#     plt.title(iweight)
#     plt.savefig("pred_{:03d}.png".format(iweight), facecolor="w")
#     plt.clf()